Parte 3 Análise de Dados com NER

Questão 3: Baixe o conjunto de dados de notícias disponível em:Folha UOL News Dataset.

* Utilize o modelo 'monilouise/ner_pt_br' para identificar e extrair entidades mencionadas nas notícias.
* Crie um ranking das organizações que mais apareceram na seção "Mercado" no primeiro trimestre de 2015.
* Apresente os resultados em um relatório detalhado, incluindo a metodologia utilizada e visualizações para apoiar a análise.

###Download de dependências

In [ ]:
libs = ["kagglehub[pandas-datasets]","wordcloud"]
try:
  !pip install -q libs
except ValueError:
  print(ValueError)

###Import das Libs

In [ ]:
import kagglehub
import pandas as pd
import numpy as np

from tqdm.notebook import tqdm


from wordcloud import WordCloud
import matplotlib.pyplot as plt



from transformers import BertForTokenClassification, DistilBertTokenizerFast, pipeline


In [ ]:
tqdm.pandas()

#### Adquirindo o dataset em ambiente colab

In [ ]:
# Download latest version
path = kagglehub.dataset_download("marlesson/news-of-the-site-folhauol", path='.')

print("Path to dataset files:", path)

In [ ]:
# Set the path to the file you'd like to load
file_path = "/kaggle/input/news-of-the-site-folhauol/articles.csv"

df = pd.read_csv(file_path)

print("First 5 records: \n")
display(df.head())

In [ ]:
df.shape

####Import do toolkit de nlp

In [ ]:
import nltk
nltk.download('stopwords')

from nltk.corpus import stopwords

# Lista em português
stopwords_pt = set(stopwords.words('portuguese'))

#####Eliminando registros nulos da coluna text, que é objeto de análise

Verificando condições do dataset

In [ ]:
df = df.dropna(axis=0, subset=['text'])


In [ ]:
df.info()

In [ ]:
df.subcategory.value_counts()

Inicio da Filtragem pelos requisitos de:



*   Notícias da Categoria de Mercado
*   Notícias do Primeiro Trimestre de 2015



In [ ]:
# Convert 'timestamp' column to datetime objects
df['timestamp'] = pd.to_datetime(df['date'])
df.shape

In [ ]:

# Filter by 'category' or 'subcategory' containing 'mercado' (case-insensitive)
df = df[df['category'].str.contains('mercado', case=False, na=False) |
                df['subcategory'].str.contains('mercado', case=False, na=False)]

df.shape

In [ ]:
df = df[
    (df['timestamp'].dt.year == 2015) &
    (df['timestamp'].dt.quarter == 1)
]
df.shape

####Instanciação do Modelo monilouise/ner_pt_br do repositório Hugging Face

Etapa envolve:



*   Tokenização
*   Modelo utilizado



In [ ]:
model = BertForTokenClassification.from_pretrained('monilouise/ner_pt_br')
tokenizer = DistilBertTokenizerFast.from_pretrained('neuralmind/bert-base-portuguese-cased'
                                                    , model_max_length=512
                                                    , do_lower_case=False
                                                    )
nlp = pipeline('ner', model=model, tokenizer=tokenizer, grouped_entities=True)

####Criação de uma coluna "doc" para ajudar na análise e evitar reprocessamentos

In [ ]:
def run_nlp(text):
    output = nlp(text)
    return output if output else []

df['doc'] = df['text'].progress_apply(run_nlp)

Análise de um dos resultados da aplicação do pipeline NLP

In [ ]:
df.head(1).doc.values[0][1]

Extração de Conteúdos direcionados

Boa parte dessas operações nem ajudou de fato na análise, mas implementei por que o tempo de execução desse código chegou a bater alarmantes 55 minutos. Em certa parte, por limitação da ferramenta do Colab, precisei usar CPU, o que ainda prejudicou mais o desempenho.

In [ ]:
#Extração de nomes de Organização
def extract_words(text):
    if text is None:
        return None
    else:
        words = [json['word'] for json in text]

        return words

df['words'] = df['doc'].progress_apply(extract_words)

In [ ]:
#Extração de Classificação das Organizações
def extract_entities(text):
    if text is None:
        return None
    else:
        entities = [json['entity_group'] for json in text]

        return entities

df['entities'] = df['doc'].progress_apply(extract_entities)

In [ ]:
#Extração de Classificação das Organizações em formato menos prolixo
def extract_entities_classifications(text):
    if text is None:
        return None
    else:
        entities = [{'word':json['word'],
                     'entity_group':json['entity_group'],
                     'score':float(json['score'])

                     } for json in text]

        return entities

df['entities_classifications'] = df['doc'].progress_apply(extract_entities_classifications)

In [ ]:
df.head(1).entities_classifications.values[0]

####Definição de Stopwords e limpeza de "spam"

In [ ]:
sw = set(list(stopwords_pt) + ['Nu','CN','BM &','FBovespa','Banco','B','Investimentos','Galvão','Brasil','Folha',"Standard & Poor ' s",'Vivo','Claro','Oi','Pactual','Times'])

In [ ]:
def reconstruir_entidades(ents, tipo='ORG'):
    entidades_reconstruidas = []
    entidade_atual = ''

    for ent in ents:
        palavra = ent['word'].strip()
        if palavra in sw:
            continue

        if (palavra.startswith('##') and ent['entity_group'] == tipo) or len(palavra) == 1:

            if palavra.startswith('##') and not palavra in sw:

              entidade_atual += palavra[2:]

            else:
              if not palavra in sw:
                entidade_atual += ' ' + palavra


        elif ent['entity_group'] == tipo:

            if entidade_atual:
                if not entidade_atual in sw and len(entidade_atual) > 2:

                  entidades_reconstruidas.append(entidade_atual)

            entidade_atual = palavra

      # salva a última
    if entidade_atual:

          entidades_reconstruidas.append(entidade_atual)

    return entidades_reconstruidas


#Técnica para Reconstrução de Orgs completas para análise

df['orgs_reconstruidas'] = df['doc'].apply(reconstruir_entidades)


In [ ]:
#Count elementar com itens da lista
from collections import Counter

counter:list =  []

for org in df.orgs_reconstruidas:
   counter.extend(org)


org_count = Counter(counter).most_common(20)
for org in org_count:
  print(org)

Durante a análise, o mais dificil foi realmente determinar as organizações. O tokenizador pareceu atuar de forma errática algumas vezes por provavelmente não ter sido treinado com vários dos termos que "encontramos" aqui. No fim, minha percepção é que muitas vezes, é preciso estudar sobre a elaboração do modelo para poder empregar ele corretamente. Possivelmente, um fine-tunning em cima desse que a autora implementou.

In [ ]:
#Plotando as Orgs
org_names, org_counts = zip(*org_count)

plt.figure(figsize=(12, 8))
plt.barh(org_names[::-1], org_counts[::-1], color='green')
plt.xlabel('Contagem')
plt.ylabel('Organização')
plt.title('Top 20 Organizações mais frequentes em artigos do 1º Trimestre de 2015')
plt.tight_layout()
plt.show()

In [ ]:
# Combine all reconstructed organizations into a single string
all_orgs_text = ' '.join(org_names)

# Create a WordCloud object
wordcloud = WordCloud(width=800, height=400, background_color='white',collocations=False).generate(all_orgs_text)

# Display the word cloud
plt.figure(figsize=(12, 6))
plt.imshow(wordcloud, interpolation='bilinear')
plt.axis('off')
plt.title('Nuvem de Palavras das Organizações')
plt.show()
